In [1]:
import numpy as np
import os
import cv2
import time
from tqdm.auto import tqdm

c:\Users\jasso\.conda\envs\computer-vision-2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
names = ['1 RIBU ASLI', '1 RIBU PALSU', '10 RIBU ASLI', '10 RIBU PALSU', 
         '100 RIBU ASLI', '100 RIBU PALSU', '20 RIBU ASLI', '20 RIBU PALSU', 
         '5 RIBU ASLI', '5 RIBU PALSU', '50 RIBU ASLI', '50 RIBU PALSU']

In [3]:
dataset_path = "C:\Codes\Computer Vision\Project\PROJECT-DATA-UANG-11"

In [4]:
def load_dataset_with_folder_structure(folder_path,augment=True):
    images_folder = os.path.join(folder_path, "images")
    labels_folder = os.path.join(folder_path, "labels")
    
    data = []
    labels = []
    count=0
    
    for img_name in os.listdir(images_folder):
        if img_name.endswith(".jpg"): 
            img_path = os.path.join(images_folder, img_name)
            label_path = os.path.join(labels_folder, os.path.splitext(img_name)[0] + ".txt")  # File label

            img = cv2.imread(img_path)
            if img is not None and os.path.exists(label_path):  # Pastikan gambar dan label ada
                resized_img = cv2.resize(img, (512, 512))  # Resize gambar
                data.append(resized_img)
                
                with open(label_path, "r") as f:
                    first_line = f.readline().strip()  # Ambil baris pertama
                    class_index = int(first_line.split()[0])  # Digit pertama sebagai label
                    labels.append(class_index)
    
    
    return np.asarray(data, dtype=np.uint8), np.array(labels, dtype=np.int32)

In [5]:
x_val, y_val=load_dataset_with_folder_structure(dataset_path+'/valid/', False)
x_test, y_test=load_dataset_with_folder_structure(dataset_path+'/test/', False)
x_train, y_train=load_dataset_with_folder_structure(dataset_path+'/train/', False)

In [6]:
import torch
from torchvision import models

In [7]:
class VGG19(torch.nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.vgg19.fc = torch.nn.Linear(in_features=2048, out_features=12)
        # self.resnet.compile()
        
    def forward(self, x):
        return self.vgg19(x)

In [8]:
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [9]:
class MoneyDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        image = self.x[idx]
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)
            label = torch.tensor(label)

        return image, label

In [10]:
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = MoneyDataset(x_train, y_train, transform)
val_dataset = MoneyDataset(x_val, y_val, transform)
test_dataset = MoneyDataset(x_test, y_test, transform)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [12]:
test_loader = DataLoader(test_dataset)

In [13]:
len(train_loader)

639

In [14]:

def run_one_epoch(
    epoch: int,
    ds_sizes,
    dataloaders,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    loss: nn.Module,
    scheduler: torch.optim.lr_scheduler,
    criterion: nn.Module
):
    global best_AUROC
    
    metrics = {}
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    for phase in ["train", "val"]:

        if phase == "train":
            model.train()
        else:
            model.eval()
        
        avg_loss = 0

        for images, labels in tqdm(dataloaders[phase], total=len(dataloaders[phase])):
            labels = labels.type(torch.LongTensor)
            images = images.to(device)
            labels = labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Track history if in phase == "train"
            with torch.set_grad_enabled(phase == "train"):
                outputs = model(images)
                
                # print(outputs)
                # print(labels)
                loss = criterion(outputs, labels)
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()

            avg_loss += loss.item() * images.size(0)

        epoch_loss = avg_loss / ds_sizes[phase]
        
        # step the scheduler
        if phase == "train":
            scheduler.step(epoch_loss)
        

        # Metrics tracking
        if phase == "train":
            metrics["train_loss"] = round(epoch_loss, 3)
        else:
            metrics["val_loss"] = round(epoch_loss, 3)

    return metrics

In [15]:
def train(dataloaders, ds_sizes, model, optimizer, criterion, scheduler):
    for epoch in range(20):
        start = time.time()

        metrics = run_one_epoch(
            epoch=epoch,
            ds_sizes=ds_sizes,
            dataloaders=dataloaders,
            model=model,
            optimizer=optimizer,
            loss=criterion,
            scheduler=scheduler,
            criterion=criterion
        )

        end = time.time() - start

        print(f"Epoch {epoch+1} completed in: {round(end/60, 3)} mins, val_loss: {metrics['val_loss']}, train_loss: {metrics['train_loss']}")
    return model

In [16]:
model = VGG19().to('cuda')
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

dataloaders = {
    "train": train_loader,
    "val": val_loader,
    "test": test_loader
}

ds_sizes = {
    "train": len(train_dataset),
    "val": len(val_dataset),
    "test": len(test_dataset)
}


train(dataloaders, ds_sizes, model, optimizer, criterion, scheduler)

c:\Users\jasso\.conda\envs\computer-vision-2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jasso\.conda\envs\computer-vision-2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 137/137 [00:14<00:00,  9.41it/s]


Epoch 1 completed in: 5.403 mins, val_loss: 2.458, train_loss: 2.779


100%|██████████| 137/137 [00:14<00:00,  9.41it/s]


Epoch 2 completed in: 5.269 mins, val_loss: 2.438, train_loss: 2.431


100%|██████████| 137/137 [00:14<00:00,  9.44it/s]


...

100%|██████████| 137/137 [00:12<00:00, 10.84it/s]


Epoch 19 completed in: 4.762 mins, val_loss: 2.414, train_loss: 2.404


100%|██████████| 137/137 [00:12<00:00, 10.84it/s]

Epoch 20 completed in: 4.761 mins, val_loss: 2.412, train_loss: 2.404


VGG19(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): Co

In [17]:
model.eval()

VGG19(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): Co

In [18]:
#predict
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f"predicted: {predicted}, actual: {labels}")

print(correct)
print(total)
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device='cuda:0'), actual: tensor([4], device='cuda:0', dtype=torch.uint8)
predicted: tensor([4], device=

In [19]:
torch.save(model.state_dict(), 'model.pth')

: 